In [ ]:
# Install required packages in the Colab runtime
!pip install pdf2image reportlab > /dev/null
!apt install poppler-utils > /dev/null
!pip install daltonlens > /dev/null
print("Successfully installed all required packages!")

In [ ]:
import numpy as np
from pdf2image import convert_from_path
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from PIL import Image
from daltonlens import convert, simulate, generate
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def simulate_colorblindness(image, cb_type="deuteranopia", severity=1.0):
    """Apply colorblind filter to an image."""
    cb_filters = {
        "deuteranopia": simulate.Deficiency.DEUTAN,
        "protanopia": simulate.Deficiency.PROTAN,
        "tritanopia": simulate.Deficiency.TRITAN,
    }
    image = image.convert('RGB')
    # limit size
    MAX_WIDTH = 1000
    MAX_HEIGHT = 2000
    if image.width > MAX_WIDTH:
        image = image.resize((MAX_WIDTH, int(image.height * MAX_WIDTH / image.width)))
    if image.height > MAX_HEIGHT:
        image = image.resize((int(image.width * MAX_HEIGHT / image.height), MAX_HEIGHT))

    im = np.asarray(image.convert('RGB'))
    if cb_type in cb_filters:
      simulator = simulate.Simulator_Machado2009()
      protan_im = simulator.simulate_cvd(im, simulate.Deficiency.DEUTAN, severity=severity)
      return Image.fromarray(protan_im)
    # elif cb_type == "grayscale":
    #   gray_im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    #   return Image.fromarray(gray_im)
    else:
      raise ValueError(f"Unsupported colorblindness type: {cb_type}")

In [ ]:
def process_images(input_files, cb_type="deuteranopia", severity=1.0, pdf_page=1):
    """Process various image formats and save as a PDF."""
    images = []
    for file in input_files:
        if file.lower().endswith(".pdf"):
            images.extend(convert_from_path(file, first_page=pdf_page, last_page=pdf_page))
        else:
            images.append(Image.open(file))
    filtered_images = [simulate_colorblindness(img, cb_type, severity) for img in images]
    return filtered_images


In [ ]:
def upload_files():
    upload_widget = widgets.FileUpload(accept='.pdf,.jpg,.jpeg,.png,', multiple=True)
    display(upload_widget)
    return upload_widget

# Upload your figure here

In [ ]:
upload_widget = upload_files()

In [ ]:
if upload_widget.value:
    uploaded_files = []
    for uploaded_filename in upload_widget.value.keys():
        with open(uploaded_filename, "wb") as f:
            f.write(upload_widget.value[uploaded_filename]['content'])
        uploaded_files.append(uploaded_filename)

    ## Change settings here
    output_pdf = "output.pdf"
    cb_type = "deuteranopia"  # Deuteranopia is the most common form of colorblindness. Try changing to 'protanopia', 'tritanopia', or 'grayscale'
    severity = 1.0            # Choose in [0,1]
    pdf_page = 7              # If you uploaded a PDF with multiple pages, select which page you would like to display

    filtered_images = process_images(uploaded_files, cb_type, severity, pdf_page)

    # Display modified images in notebook
    for img in filtered_images:
        display(img)